In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/conda/bin/python'

In [2]:
import datalabframework as dlf
import pyspark.sql.functions as F
import pyspark.sql.types as T
import yaml
import pandas as pd
from datetime import datetime, timedelta
from util_report.util_reports import *
from util_report.util_email import *

In [3]:
pd.set_option('display.max_columns', 30)

### Setup engine

In [4]:
dlf.project.load('default')
engine = dlf.project.engine()
spark = engine.context() 

AttributeError: 'StringDumpYAML' object has no attribute 'YAMLError'

### Setup date

In [5]:
# md = dlf.project.metadata()
# report_date_str = md['variables']['report_date']
report_date_str = "2019-06-05"
format_str= "%Y-%m-%d"
report_date = datetime.strptime(report_date_str, format_str)
cube_date = report_date - timedelta(days=1)
first_of_last_month = (report_date.replace(day=1) - timedelta(days=1)).replace(day=1).replace(hour=0)
lastweek = (report_date - timedelta(days=7))
this_month = (report_date - timedelta(days=1)).month
this_year = (report_date - timedelta(days=1)).year

### Get config

In [ ]:
config = yaml.load(open('config/config.yml'))

In [ ]:
pd_df = None

In [ ]:
highlight_index = {}
headers = {}

In [ ]:
report_jinja = get_data_before_render(
    config,
    pd_df,
    report_date,
    headers,
    highlight_index
)

In [ ]:
content = ""
content += render_content('header.html', {
    'style': get_styles('eod.yml'), 
    'data': {
        'title': 'some title',
        'extras_information': {
            'info1': 'something',
            'info2': 'something again'
        }
    }
})
content += render_content('table.html', {'data': pd_df})

In [ ]:
html = ''.join(content)
from yattag import indent
html_pretty = indent(
    html,
    indentation = '    ',
    newline = '\r\n',
    indent_text = True
)
#Write to file 
f = open('eod_new.html', "wb")
f.write(str(html_pretty).encode('utf-8'))
f.close()
print("write done")
